In [2]:
#With credits to Panos Athanasiou & Tim Wilde (30/11/2022)
#Produced during first Beira Hackaton

import pandas as pd
import geopandas as gpd
import numpy as np
from matplotlib import pyplot as plt
import matplotlib
import xarray as xr
import scipy.interpolate
from scipy.interpolate import griddata
import contextily
#from osgeo import osr
#from osgeo import ogr
#from osgeo import gdal
import rasterio
from rasterio.transform import from_origin

def write_raster(fn, x, y, data, driver='GTiff', crs=3857, compress='lzw', center=False, nodata=-9999.):
    res = (x[-1] - x[0]) / float(len(x) - 1)
    crs = rasterio.crs.CRS({"init": "epsg:{}".format(crs)})
    if not center:
        transform = from_origin(x[0], y[-1] + res, res, res)
    else:
        transform = from_origin(x[0] - res / 2., y[-1] + res / 2., res, res)
    with rasterio.open(fn, 'w', driver=driver, nodata=nodata,
                       height=data.shape[0], width=data.shape[1],
                       count=1, dtype=data.dtype,
                       crs=crs, transform=transform,
                       compress=compress) as dst:
        dst.write(data, 1)


In [3]:
# Input
method = 'nearest'
crs = 32736
sfincs_output = '../examples/sfincs_beira_map.nc'
#Use 20 m resolution DEM file to use as grid for sfincs output
dem_path = '../examples/topobathy_new.tiff'


In [4]:
# Open sfincs data and get node centres
ds = xr.open_dataset(sfincs_output)
xn, yn = ds['mesh2d_node_x'].values, ds['mesh2d_node_y'].values
nodes = np.asarray(ds['mesh2d_face_nodes'].values, int)-1
x, y = np.average(xn[nodes], axis=1), np.average(yn[nodes], axis=1)
zs = np.squeeze(ds['zsmax'].values)
zb = np.squeeze(ds['zb'].values)

In [5]:
# Open DEM file
dem = xr.open_rasterio(dem_path)
x0, y0 = dem['x'].values, dem['y'].values[::-1]
elv = dem.values[0, : , :]
elv = np.where(elv == dem.attrs['nodatavals'][0], np.nan, elv)

C:\Users\weeber\AppData\Local\Temp\ipykernel_7252\2652016603.py:2: DeprecationWarning: open_rasterio is Deprecated in favor of rioxarray. For information about transitioning, see: https://corteva.github.io/rioxarray/stable/getting_started/getting_started.html
  dem = xr.open_rasterio(dem_path)


In [6]:
#create 2D arrays
xx, yy = np.meshgrid(x0, y0)

In [7]:
# Reinterpolate SFINCS max water levels to the DEM grid
grid_zs_max = np.flipud(griddata(np.stack((x, y), axis=1), zs, (xx, yy), method=method))

In [8]:
# translate to flood depths
fd_max = grid_zs_max - elv 
fd_max = np.where(fd_max<0.05, np.nan, fd_max) #minimum flood depth of at least 5 cm
grid_fd_max = np.flipud(np.reshape(fd_max,(y0.size,x0.size)))

In [9]:
#write as tif files
print("Writing max flood depth to tif files")

fn_out = sfincs_output.replace('_map.nc','_max_flood_depth')
fn_out_wl = sfincs_output.replace('_map.nc','_max_water_level')
fn_out_duration = sfincs_output.replace('_map.nc','_flood_duration')
fn_out_ts = sfincs_output.replace('_map.nc','_flood_depth')
fn_out_nc = sfincs_output.replace('_map.nc','_flood_depth_data')

write_raster(fn_out_wl+'.tif', x0, y0, grid_zs_max, driver='GTiff', crs=crs, center=True, nodata=-9999)
write_raster(fn_out+'.tif', x0, y0, fd_max, driver='GTiff', crs=crs, center=True, nodata=-9999)

print("Calculation of max flood depth finished.")
print("Continuing with time series...")

Writing max flood depth to tif files
Calculation of max flood depth finished.
Continuing with time series...


In [10]:
#Extract data and calculate flooddepth for every time step on detailed grid
time = ds['time'].values
dt_filter = 8 #write flood maps every 24 hrs (8*3 = 24 hr)

In [11]:
#create empty array for data 
data_tot = np.zeros((y0.size,x0.size,len(time)))
data_wl_tot = np.zeros((y0.size,x0.size,len(time)))
for ii in range(0,len(time)):
    print('Work on time step '+str(ii+1)+' of '+str(len(time)))
    zs = np.squeeze(ds['zs'].values[ii])
    grid_zs = np.flipud(griddata(np.stack((x, y), axis=1), zs, (xx, yy), method=method))
    fd = grid_zs - elv
    grid_fd = np.flipud(np.reshape(fd,(y0.size,x0.size)))
    data_tot[:,:,ii] = grid_fd
    data_wl_tot[:,:,ii] = grid_zs
    if ii % dt_filter == 0:
        print("Write 24 hrs output to tif file (time step "+str(ii+1)+" of "+str(len(time))+")")
        dt = str(time[ii])
        write_raster(fn_out_ts+dt[0:13]+'.tif', x0, y0, fd, driver='GTiff', crs=crs, center=True, nodata=-9999)

Work on time step 1 of 65
Write 24 hrs output to tif file (time step 1 of 65)
Work on time step 2 of 65
Work on time step 3 of 65
Work on time step 4 of 65
Work on time step 5 of 65
Work on time step 6 of 65
Work on time step 7 of 65
Work on time step 8 of 65
Work on time step 9 of 65
Write 24 hrs output to tif file (time step 9 of 65)
Work on time step 10 of 65
Work on time step 11 of 65
Work on time step 12 of 65
Work on time step 13 of 65
Work on time step 14 of 65
Work on time step 15 of 65
Work on time step 16 of 65
Work on time step 17 of 65
Write 24 hrs output to tif file (time step 17 of 65)
Work on time step 18 of 65
Work on time step 19 of 65
Work on time step 20 of 65
Work on time step 21 of 65
Work on time step 22 of 65
Work on time step 23 of 65
Work on time step 24 of 65
Work on time step 25 of 65
Write 24 hrs output to tif file (time step 25 of 65)
Work on time step 26 of 65
Work on time step 27 of 65
Work on time step 28 of 65
Work on time step 29 of 65
Work on time ste

In [12]:
# Calculate flood duration above a certian threshold based on detailed bathymetry
threshold = 0.1 #m
flooded = data_tot>=threshold 
timestep_map_file = 3 #[hrs]
duration = np.sum(flooded,axis=2)*timestep_map_file #duration in [hrs]

print("Write data to netcdf file...")

ds_crs = ds['crs'].copy()

ds_ts = xr.Dataset(
    data_vars=dict(
        flooddepth =(["xcoord", "ycoord","time"], data_tot[:,:,1:dt_filter:len(time)]),
        waterlevel =(["xcoord", "ycoord","time"], data_wl_tot[:,:,1:dt_filter:len(time)]),
        maxflooddepth = (["xcoord", "ycoord"], grid_fd_max),
        maxwaterlevel = (["xcoord", "ycoord"], grid_zs_max),
        duration = (["xcoord", "ycoord"], duration),
    ),
    coords=dict(
        x=(["xcoord", "ycoord",], xx),
        y=(["xcoord", "ycoord",], yy),
        time= time[1:dt_filter:len(time)], 
    ),
    attrs=dict(
        description="Flood depth data.",
        units="m"),
)

ds_ts = ds_ts.merge(ds_crs)
ds_ts['crs'] = ds.crs.assign_attrs(epsg_code='EPSG:'+str(crs))
ds_ts['crs'].values = crs
ds_ts.to_netcdf(path=fn_out_nc+'.nc',mode='w')


Write data to netcdf file...


In [13]:
#Write duration to tif file
write_raster(fn_out_duration+'.tif', x0, y0, np.flipud(duration), driver='GTiff', crs=crs, center=True, nodata=-9999)